In [1]:
import json


def read_file(path):
    return json.load(open(path))

In [2]:
file1 = read_file('file1.json')
file2 = read_file('file2.json')

In [615]:
file1 = {
  "host": "hexlet.io",
  "timeout": 50,
  "proxy": "123.234.53.22",
  "follow": 'false'
}

file2 = {
  "timeout": 20,
  "verbose": 'true',
  "host": "hexlet.io"
}

In [616]:
a = {
  "common": {
    "setting1": "Value 1",
    "setting2": 200,
    "setting3": True,
    "setting6": {
      "key": "value",
      "doge": {
        "wow": ""
      }
    }
  },
  "group1": {
    "baz": "bas",
    "foo": "bar",
    "nest": {
      "key": "value"
    }
  },
  "group2": {
    "abc": 12345,
    "deep": {
      "id": 45
    }
  }
}
b = {
  "common": {
    "follow": False,
    "setting1": "Value 1",
    "setting3": "Null",
    "setting4": "blah blah",
    "setting5": {
      "key5": "value5"
    },
    "setting6": {
      "key": "value",
      "ops": "vops",
      "doge": {
        "wow": "so much"
      }
    }
  },
  "group1": {
    "foo": "bar",
    "baz": "bars",
    "nest": "str"
  },
  "group3": {
    "deep": {
      "id": {
        "number": 45
      }
    },
    "fee": 100500
  }
}

In [617]:
REMOVED = 'removed'
ADDED = 'added'
UNMODIFIED = 'unmodified'
MODIFIED = 'modified'
NESTED = 'nested'


def generate_diff(file_before, file_after):
    result = {}
    
    united_keys = file_before.keys() & file_after.keys()
    removed_keys = file_before.keys() - file_after.keys()
    added_keys = file_after.keys() - file_before.keys()
    
    #check removed keys
    for key in removed_keys:
        if isinstance(file_before[key], dict):
            result[key] = (REMOVED, generate_diff(file_before[key], file_before[key]))
        else:
            result[key] = (REMOVED, file_before[key])
    
    #check added keys
    for key in added_keys:
        if isinstance(file_after[key], dict):
            result[key] = (ADDED, generate_diff(file_after[key], file_after[key]))
        else:
            result[key] = (ADDED, file_after[key])
    
    #check modified and unmodified keys
    for key in united_keys:
        if file_before[key] == file_after[key]:
            if isinstance(file_before[key], dict):
                result[key] = (UNMODIFIED, generate_diff(file_before[key], file_before[key]))
            else:
                result[key] = (UNMODIFIED, file_before[key])
        else:
            if isinstance(file_before[key], dict) and isinstance(file_after[key], dict):
                result[key] = (NESTED, generate_diff(file_before[key], file_after[key]))
            else:
                result[key] = (MODIFIED, (file_before[key], file_after[key]))
    return result

generate_diff(file1, file2)

{'follow': ('removed', 'false'),
 'proxy': ('removed', '123.234.53.22'),
 'verbose': ('added', 'true'),
 'timeout': ('modified', (50, 20)),
 'host': ('unmodified', 'hexlet.io')}

In [618]:
generate_diff(a, b)

{'group2': ('removed',
  {'deep': ('unmodified', {'id': ('unmodified', 45)}),
   'abc': ('unmodified', 12345)}),
 'group3': ('added',
  {'fee': ('unmodified', 100500),
   'deep': ('unmodified',
    {'id': ('unmodified', {'number': ('unmodified', 45)})})}),
 'group1': ('nested',
  {'nest': ('modified', ({'key': 'value'}, 'str')),
   'foo': ('unmodified', 'bar'),
   'baz': ('modified', ('bas', 'bars'))}),
 'common': ('nested',
  {'setting2': ('removed', 200),
   'follow': ('added', False),
   'setting4': ('added', 'blah blah'),
   'setting5': ('added', {'key5': ('unmodified', 'value5')}),
   'setting3': ('modified', (True, 'Null')),
   'setting1': ('unmodified', 'Value 1'),
   'setting6': ('nested',
    {'ops': ('added', 'vops'),
     'key': ('unmodified', 'value'),
     'doge': ('nested', {'wow': ('modified', ('', 'so much'))})})})}

# Print string

In [619]:
from collections import OrderedDict

states = {'removed': '- ', 
          'added': '+ ',
          'unmodified': '  ',
          'nested': '  '}

def check_status(key, state, value, shift = ' '):
    if state == 'modified':
        print(shift + '- ' + str( key)+ ': '+  str(value[0]))
        print(shift + '+ ' + str( key)+ ': '+  str(value[1]))
    else:
        print(shift + states[state] + str( key)+ ': '+  str(value))

def parse_dict(answer):
    answer = OrderedDict(sorted(answer.items()))
    shift = ' '
    for key, (state, value) in answer.items():
        i=+2
        if not isinstance(value, dict):
            check_status(key, state, value, shift = i*shift)
        else:
            
            print(i*shift + states[state] + str( key)+ ': {')
            parse_dict_3(value, i*shift)
            print(i*shift + ' }')
    return  

In [620]:
parse_dict(generate_diff(a, b))

    common: {
    + follow: False
      setting1: Value 1
    - setting2: 200
    - setting3: True
    + setting3: Null
    + setting4: blah blah
    + setting5: {
          key5: value5
     }
      setting6: {
          doge: {
                - wow: 
                + wow: so much
         }
          key: value
        + ops: vops
     }
   }
    group1: {
    - baz: bas
    + baz: bars
      foo: bar
    - nest: {'key': 'value'}
    + nest: str
   }
  - group2: {
      abc: 12345
      deep: {
          id: 45
     }
   }
  + group3: {
      deep: {
          id: {
                  number: 45
         }
     }
      fee: 100500
   }


# Parse plain

In [621]:
import numpy as np

In [648]:
def check_status(path, key, state, value):
    path_full = "{}.{}".format(path, key)
    if state == 'modified':
        line = """Property '{}' was updated. From '{}' to '{}'""".format(path_full, value[0], value[1])
    elif state == 'removed':
        line = """Property '{}' was '{}'""".format(path_full, state)
    elif state == 'added':
        line = """Property '{}' was '{}' with value: '{}'""".format(path_full, state, value)
    else:
        line = ''
    return line

def parse_plain(answer, path = ''):
    answer = OrderedDict(sorted(answer.items()))
    for key, (state, value) in answer.items():
        if not isinstance(value, dict):
            path = "{}.{}".format(path, key)
            line = check_status(path, key, state, value)
            print(line)
        else:
            path = "{}.{}".format(path, key)
            parse_plain(value, path)
                
    return 

In [649]:
parse_plain(generate_diff(a, b))

Property '.common.follow.follow' was 'added' with value: 'False'

Property '.common.follow.setting1.setting2.setting2' was 'removed'
Property '.common.follow.setting1.setting2.setting3.setting3' was updated. From 'True' to 'Null'
Property '.common.follow.setting1.setting2.setting3.setting4.setting4' was 'added' with value: 'blah blah'

Property '.common.follow.setting1.setting2.setting3.setting4.setting5.setting6.doge.wow.wow' was updated. From '' to 'so much'

Property '.common.follow.setting1.setting2.setting3.setting4.setting5.setting6.doge.key.ops.ops' was 'added' with value: 'vops'
Property '.common.group1.baz.baz' was updated. From 'bas' to 'bars'

Property '.common.group1.baz.foo.nest.nest' was updated. From '{'key': 'value'}' to 'str'






In [633]:
generate_diff(a, b)

{'group2': ('removed',
  {'deep': ('unmodified', {'id': ('unmodified', 45)}),
   'abc': ('unmodified', 12345)}),
 'group3': ('added',
  {'fee': ('unmodified', 100500),
   'deep': ('unmodified',
    {'id': ('unmodified', {'number': ('unmodified', 45)})})}),
 'group1': ('nested',
  {'nest': ('modified', ({'key': 'value'}, 'str')),
   'foo': ('unmodified', 'bar'),
   'baz': ('modified', ('bas', 'bars'))}),
 'common': ('nested',
  {'setting2': ('removed', 200),
   'follow': ('added', False),
   'setting4': ('added', 'blah blah'),
   'setting5': ('added', {'key5': ('unmodified', 'value5')}),
   'setting3': ('modified', (True, 'Null')),
   'setting1': ('unmodified', 'Value 1'),
   'setting6': ('nested',
    {'ops': ('added', 'vops'),
     'key': ('unmodified', 'value'),
     'doge': ('nested', {'wow': ('modified', ('', 'so much'))})})})}